In [34]:
from ansys.mapdl.core.errors import MapdlRuntimeError
from ansys.mapdl.core import launch_mapdl
import re
import os
import pandas as pd
pd.set_option('display.max_colwidth', 150)

In [13]:
source_dir = 'D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess'

subdir_arr = []
file_name_arr = []
complete_path_arr = []

# Lista de subdiretórios
for subdir in os.listdir(source_dir):
    dir_path = os.path.join(source_dir, subdir)
    print(dir_path)
    if os.path.isdir(dir_path):  # Verifica se é um diretório
        subdir_arr.append(dir_path)  # Adiciona à lista

print('\n')
# Extrair o nome da última pasta do primeiro subdiretório
for subdir in subdir_arr:
    file_name = os.path.basename(subdir)
    print(file_name)
    file_name_arr.append(file_name)

print('\n')
for subdir in subdir_arr:
    file_name = os.path.basename(subdir) + '.db'
    complete_path = os.path.join(subdir, file_name)
    print(complete_path)
    complete_path_arr.append(complete_path)


D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L2T2_k3.55_MS25_SP5
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T4_k0.9_MS25_SP55
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T5_k0.766_MS25_SP63
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k1.4_MS25_SP78
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k2.2_MS25_SP77
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T5_k0.4_MS25_SP97
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L5T5_k0.485_MS25_SP128


BL2M1P0.3L2T2_k3.55_MS25_SP5
BL2M1P0.3L3T4_k0.9_MS25_SP55
BL2M1P0.3L3T5_k0.766_MS25_SP63
BL2M1P0.3L4T3_k1.4_MS25_SP78
BL2M1P0.3L4T3_k2.2_

In [14]:
material_yielding_stress = 355
p_u = 3479
MAPDL_RUN_LOCATION='D:/constructal_automate_analysis_files'
MAPDL_START_TIMEOUT=120

In [42]:
mapdl = launch_mapdl(
    port=50052,
    run_location=MAPDL_RUN_LOCATION,
    nproc=4,
    override=True,
    loglevel="INFO",
    start_timeout=MAPDL_START_TIMEOUT,
    cleanup_on_exit=True
)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_ana

In [35]:
def resume_mapdl_analysis(analysis_dir, analysis_file_name, mapdl):
    print(f'Dir: {analysis_dir}. Filename: {analysis_file_name}')
    mapdl.cwd(analysis_dir)
    mapdl.filname(fname=analysis_file_name, key=0)
    mapdl.resume(fname=analysis_file_name, ext = 'db')

In [36]:
def calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u):
    first_to_last_arr_position = len(mapdl.post_processing.frequency_values)-2
    first_to_last_load = mapdl.post_processing.frequency_values[first_to_last_arr_position]

    second_to_last_arr_position = len(mapdl.post_processing.frequency_values)-3
    second_to_last_load = mapdl.post_processing.frequency_values[second_to_last_arr_position]

    diff_between_loads = first_to_last_load - second_to_last_load
    load_increment = p_u/400

    if diff_between_loads > load_increment:
        return first_to_last_arr_position
    else:
        return second_to_last_arr_position

In [37]:
def plot_images(mapdl, analysis_db_path, material_yielding_stress, p_u, position):
    VON_MISES_IMG_SUFFIX = '_von_mises_dist.png'
    W_IMG_SUFFIX = '_w_dist.png'

    von_mises_dist_img_path = analysis_db_path.replace('.db', VON_MISES_IMG_SUFFIX)
    w_dist_img_path = analysis_db_path.replace('.db', W_IMG_SUFFIX)

    mapdl.result.plot_principal_nodal_stress(
        position,
        "SEQV",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,          
        edge_color="black",       
        cmap="jet",              
        rng=[0, material_yielding_stress],
        line_width=1.0,           
        off_screen=True,
        # screenshot=von_mises_dist_img_path,
        show_scalar_bar=True,
        # show_displacement=True,
        # displacement_factor=2,
        scalar_bar_args={
            "title": "Tensões (MPa)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )

    mapdl.result.plot_principal_nodal_stress(
        position,
        "SEQV",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,          
        edge_color="black",       
        cmap="jet",              
        rng=[0, material_yielding_stress],
        line_width=1.0,           
        off_screen=True,
        screenshot=von_mises_dist_img_path,
        show_scalar_bar=True,
        # show_displacement=True,
        # displacement_factor=2,
        scalar_bar_args={
            "title": "Tensões (MPa)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )

    mapdl.result.plot_nodal_displacement(
        position,
        "UZ",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,
        edge_color="black",
        cmap="jet",
        line_width=1.0,
        off_screen=True,
        screenshot=w_dist_img_path,
        show_scalar_bar=True,
        scalar_bar_args={
            "title": "Deslocamento em Z (mm)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )
    

    return von_mises_dist_img_path, w_dist_img_path

In [38]:
def gerar_graficos_para_analises_com_problema(mapdl, analysis_dir, analysis_file_name, analysis_db_path, material_yielding_stress, p_u):
    resume_mapdl_analysis(analysis_dir, analysis_file_name, mapdl)
    position = calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u)
    plot_images(mapdl, analysis_db_path, material_yielding_stress, p_u, position)

In [30]:
for analysis_dir, analysis_file_name, analysis_db_path in zip(subdir_arr, file_name_arr, complete_path_arr):
    gerar_graficos_para_analises_com_problema(mapdl, analysis_dir, analysis_file_name, analysis_db_path, material_yielding_stress, p_u)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T4_k0.9_MS25_SP55
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T4_k0.9_MS25_SP55
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T4_k0.9_MS25_SP55
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L2T2_k3.55_MS25_SP5
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L2T2_k3.55_MS25_SP5
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L2T2_k3.55_MS25_SP5
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.

Dir: D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L2T2_k3.55_MS25_SP5. Filename: BL2M1P0.3L2T2_k3.55_MS25_SP5


Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x2892ddb7350_54&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T2_k3.55_MS25_SP5
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T2_k3.55_MS25_SP5
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T2_k3.55_MS25_SP5
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L2T2_k3.55_MS25_SP5
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L2T2_k3.55_MS25_SP5
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L2T2_k3.55_MS25_SP5


Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x288db89aa50_55&reconnect=auto" class="pyvi…

...

Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x289f48fcf80_75&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L5T5_k0.485_MS25_S

Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x289f48fd070_76&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L5T5_k0.485_MS25_S

Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a5f648da0_77&reconnect=auto" class="pyvi…

In [21]:
subdir_arr_retry = [subdir_arr[2], subdir_arr[5]]
print(subdir_arr_retry)
print('\n')
file_name_arr_retry = [file_name_arr[2], file_name_arr[5]]
print(file_name_arr_retry)
print('\n')
complete_path_arr_retry = [complete_path_arr[2], complete_path_arr[5]]
bprint(complete_path_arr_retry)

['D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\\BL2M1P0.3L3T5_k0.766_MS25_SP63', 'D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\\BL2M1P0.3L4T3_k8.6_MS25_SP75']


['BL2M1P0.3L3T5_k0.766_MS25_SP63', 'BL2M1P0.3L4T3_k8.6_MS25_SP75']


['D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\\BL2M1P0.3L3T5_k0.766_MS25_SP63\\BL2M1P0.3L3T5_k0.766_MS25_SP63.db', 'D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\\BL2M1P0.3L4T3_k8.6_MS25_SP75\\BL2M1P0.3L4T3_k8.6_MS25_SP75.db']


In [44]:
def gerar_graficos_para_analises_com_problema_retry(mapdl, analysis_dir, analysis_file_name, analysis_db_path, material_yielding_stress, p_u):
    resume_mapdl_analysis(analysis_dir, analysis_file_name, mapdl)
    #Exceção para os ids 63 e 75
    # position = calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u)-1
    position = calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u)
    plot_images(mapdl, analysis_db_path, material_yielding_stress, p_u, position)

In [31]:
for analysis_dir, analysis_file_name, analysis_db_path in zip(subdir_arr_retry, file_name_arr_retry, complete_path_arr_retry):
    gerar_graficos_para_analises_com_problema_retry(mapdl, analysis_dir, analysis_file_name, analysis_db_path, material_yielding_stress, p_u)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k1.4_MS25_SP78
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k1.4_MS25_SP78
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k1.4_MS25_SP78
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L3T5_k0.766_MS25_SP63
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L3T5_k0.766_MS25_SP63
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L3T5_k0.766_MS25_SP63
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL

Dir: D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T5_k0.766_MS25_SP63. Filename: BL2M1P0.3L3T5_k0.766_MS25_SP63


Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a5f64b1d0_78&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L3T5_k0.766_MS25_SP63
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L3T5_k0.766_MS25_SP63
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L3T5_k0.766_MS25_SP63
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T5_k0.766_MS25_SP63
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T5_k0.766_MS25_SP63
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T5_k0.766_MS25_SP63


Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a5f64b740_79&reconnect=auto" class="pyvi…

...

Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a71494bf0_81&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75


Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a7153d760_82&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75


Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a715718b0_83&reconnect=auto" class="pyvi…

In [32]:
mapdl.exit()

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3

In [45]:
analysis_dir = 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71'
analysis_file_name = 'BL2M1P0.3L4T2_k1.233_MS25_SP71'
analysis_db_path = 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71/BL2M1P0.3L4T2_k1.233_MS25_SP71.db'

material_yielding_stress = 355
p_u = 3479

gerar_graficos_para_analises_com_problema_retry(mapdl, analysis_dir, analysis_file_name, analysis_db_path, material_yielding_stress, p_u)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.

Dir: D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71. Filename: BL2M1P0.3L4T2_k1.233_MS25_SP71


Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a21a1c6b0_87&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50

Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a21a1baa0_88&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50

Widget(value='<iframe src="http://localhost:60131/index.html?ui=P_0x28a21a169f0_89&reconnect=auto" class="pyvi…